In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def squared_norm(x):
    x_val = np.float64(x[0][0])
    y_val = np.float64(x[1][0])
    return x_val*x_val + y_val*y_val

def norm(x):
    return np.sqrt(squared_norm(x))

def get_unit_vector(x):
    norm_value = norm(x)
    if norm_value == np.float64(0.0):
        return x
    return (x/norm_value).copy()

In [3]:
def f1(x):
    A = np.array([[1,0],[0,1]])
    h = np.matmul(x.T,A)
    return 0.5*np.matmul(h,x)

def f2(x):
    A = np.array([[10,8],[8,10]])
    h = np.matmul(x.T,A)
    return 0.5*np.matmul(h,x)
    
def gradient_f1(x):
    x_value = np.float64(x[0][0])
    y_value = np.float64(x[1][0])
    return np.array([[x_value],[y_value]])

def gradient_f2(x):
    x_value = np.float64(x[0][0])
    y_value = np.float64(x[1][0])
    return np.array([[10*x_value + 8 *y_value],[10*y_value + 8 * x_value]])

def hessian_f1(x):
    return np.array([[1,0],[0,1]])

def hessian_f2(x):
    return np.array([[10,8],[8,10]])



In [4]:
def steepest_descent(initial_point,function='f1'):

    #initial_point = np.array([[2.0],[0.0]],dtype=np.float64)
    fun = f1 if function == 'f1' else f2
    gradient_fun = gradient_f1 if function == 'f1' else gradient_f2

    current_point = initial_point.copy()
    
    c = 0.0001
    rho = 0.9
    iterations = 0
    iterates = []

    iterates.append(current_point)

    while True:        
        
        iterations += 1
        gradient = gradient_fun(current_point)
        p = -get_unit_vector(gradient)            
        alpha  = np.float64(1.0)
        previous_point = current_point.copy() 
        break_inner_loop = False
    
        current_function_value = np.float64(fun(current_point).squeeze()) 
        while not break_inner_loop:
            new_point = current_point + alpha*p

            new_function_value = np.float64(fun(new_point).squeeze())
            increment_value = np.float64(((c*alpha)*np.dot(gradient.T,p)).squeeze())

            if((new_function_value > (current_function_value + increment_value))):
                alpha = alpha * rho
            else:
                break_inner_loop = True
     

        new_point = current_point + alpha*p
        current_point = new_point.copy()
        iterates.append(current_point)
        if np.linalg.norm(current_point-previous_point) < 0.00001:
            break
    return iterations,iterates


In [5]:
def newtons_method(initial_point,function = 'f1'):

    fun = f1 if function == 'f1' else f2
    gradient_fun = gradient_f1 if function == 'f1' else gradient_f2
    hessian_fun = hessian_f1 if function =='f1' else hessian_f2
    current_point = initial_point.copy()
    
    iterations = 0
    iterates = []
    iterates.append(current_point)
    c = 0.0001
    rho = 0.9

    while True:        
        iterations+=1
        gradient = gradient_fun(current_point)
        hessian = hessian_fun(current_point)
        m = np.matmul(np.linalg.inv(hessian),gradient)
        p = -m
        
        break_inner_loop = False
        alpha  = np.float64(1.0)


        current_function_value = np.float64(fun(current_point).squeeze()) 
        while not break_inner_loop:
            new_point = current_point + alpha*p

            new_function_value = np.float64(fun(new_point).squeeze())
            increment_value = np.float64(((c*alpha)*np.dot(gradient.T,p)).squeeze())
            second_order_term = (c*(alpha*alpha/2)*(np.matmul(np.matmul(p.T,hessian),p))).squeeze()
            increment_value += np.float64(second_order_term)
            
            if((new_function_value > (current_function_value + increment_value))):
                alpha = alpha * rho
            else:
                break_inner_loop = True
      
        
        previous_point = current_point.copy() 
        new_point = current_point + alpha*p
        current_point = new_point.copy()
        iterates.append(current_point)
        if np.linalg.norm(current_point-previous_point) < 0.00001:
            break
    
    return iterations,iterates

In [12]:
initial_point = np.array([[150.0],[20.0]],dtype=np.float64)
newtons_method(initial_point,function='f2')

(2,
 [array([[150.],
         [ 20.]]),
  array([[0.00000000e+00],
         [7.10542736e-15]]),
  array([[3.94430453e-31],
         [0.00000000e+00]])])

### Seperator

In [ ]:
import pandas as pd

dataframe = pd.read_csv('./data/state_wise_daily.csv')

In [ ]:
summed_data = dataframe.sum(axis=1,numeric_only=True)

In [ ]:
infected_indices = [3*i for i in range(int(len(summed_data)/3))]
indices = [ 3*i+1 for i in range(int(len(summed_data)/3)) ]
indices =  indices +  [ 3*i+2 for i in range(int(len(summed_data)/3)) ]
indices = sorted(indices)


In [ ]:
recovered_deceased = summed_data[indices]
recovered_dup = recovered_deceased.groupby(recovered_deceased.index//3).transform('sum')
rec_indices = [ 3*i + 1 for i in range(int(len(recovered_dup)//2))]
recovered = recovered_dup[rec_indices]
recovered = recovered.reset_index(drop = True)


In [ ]:
infected = summed_data[infected_indices]
infected = infected.reset_index(drop=True)

In [ ]:
new_dataframe = pd.DataFrame({'infected' :  infected , 'recovered' : recovered})

In [ ]:
new_dataframe